# Data Import & Preprocessing 
### Project: Dog Breed Image Classifier 
Team Members: Taylor Witte, Donald Yu, Praveen Manimaran, Vitush Agarwal, Parker Aman
UCSD Spring 2024 232R Big Data Analytics Using Spark 


## Enviroment Setup

In [ ]:
## Dataset
path = "/kaggle/input/143-different-dog-breeds-image-classifier/pet classifer image directory"